# Ovenbird Song Recognizer
This notebook has been developed to allow users to use the Ovenbird song recognizer developed by Chris Scott and Elly Knight to identify Ovenbird songs on their own recordings.  It does not produce a binary score ("yes" or "no") for simple identification of Ovenbird presence, but rather a sigmoidal score between 0 and 1.  The user must determine a threshold at which they consider a score to be a positive identification of an Ovenbird.  See [Knight et al. 2017](https://doi.org/10.5751/ACE-01114-120214) for information on how to evaluate a model and how to select a threshold for your statistical purposes.

This notebook also provides a distance estimation function that will return an estimated distance (with 95% CI) of the Ovenbird from the ARU based on the relative sound level (volume) of the song.  Please read the description of this model and the conditions it was developed under to know the possible limitations of it before using the distance estimation function on your own recordings.

THE SOFTWARE IS PROVIDED "AS IS", UNDER THE MIT LICENSE, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

<a target="_blank" href="https://colab.research.google.com/github/ABbiodiversity/Recognizer_Ovenbird/blob/main/Ovenbird_Recognizer_Processing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Import Packages

Import packages to run the model.  Updates to packages since the model was originally created may result in warnings when running the model.  As of June 2024 the most up to date versions of the packages below allowed the model to be run successfully on audio data despite any warnings that python may generate during the output.

In [2]:
from datetime import timedelta
import librosa
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt
from scipy.special import expit
import tensorflow as tf
import time
from tqdm import tqdm_notebook

## Google Colab Steps

If running the notebook in Colab then you'll need to mount Google Drive and clone the repository there in order to call the models and your recording files when processing.  Skip the next two steps if running on a local machine.

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

### Clone the Github Repository to Google Drive

If running the notebook through Colab then clone the Github repository to your Google Drive in order to access the model files stored in the repository.

In [ ]:
!git clone https://github.com/ABbiodiversity/Recognizer_Ovenbird.git

## Change your working directory
Change your working directory to the folder containing the notebook and the model files.

### If running locally
To run this notebok on your local machine you can either clone this git repository to your local machine, or you can download all the necessary files from github to a local drive folder without cloning the repository. The working directory in either case will be the folder that contains the notebook, model files and distance prediction file.
```python
%cd "C:/Users/User/Folder/path/"
```

### If running in Colab
The location should be the location that the above step cloned into.  The output of the above step should look something like this:
```markdown
Cloning into 'Recognizer_Ovenbird'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 83 (delta 48), reused 34 (delta 14), pack-reused 0
Receiving objects: 100% (83/83), 544.00 KiB | 9.71 MiB/s, done.
Resolving deltas: 100% (48/48), done.
```
In this instance the github repository was cloned into the location 'Recognizer_Ovenbird'.  Therefore you will change the working directory to match that location:
```python
%cd Recognizer_Ovenbird
```
Note, no additional folder path information (i.e. 'content/drive/MyDrive/') which may be necessary for other Google Drive location mapping is necessary for this step when using a repository cloned to Google Drive.

In [ ]:
# Change the working directory below to the appropriate directory for your files
%cd "C:/Users/User/Documents/Repos/Recognizer_Ovenbird"

## Designate Recording Folder

Enter the full filepath of the folder where the recording files to be processed are stored. The filepath will need to be changed to wherever you have stored the files on your machine or on Google Drive, depending on your processing choice.

Example filepath for local files: 
```python
'C:/Users/User/Documents/Ovenbird_recordings/'
```
If you're unfamiliar with the syntax for Google Drive paths in Colab, "/content/drive/" is required, followed by the rest of the filepath in your Google Drive folder structure, excluding any spaces in the folder names. Example filepath for Google Drive folder: 
```python
'/content/drive/MyDrive/Ovenbird/Recordings_Ovenbird/'
```

In [28]:
# Change path below to match your recordings folder filepath
RECORDINGS = '/content/drive/MyDrive/Ovenbird/Recordings_Ovenbird/'

### Check the recording files
If you want to check that you have correctly designated the folderpath of your target recordings and that the folder contains the recording files then you can run the following cell to list all the contents of your RECORDINGS folder.

In [ ]:
os.listdir(RECORDINGS)

## Designate the results folder
This will create a folder within the RECORDINGS folder that will contain the results of the recognizer. This folder wil include the date of the model run to help distinguish different runs of the model.

In [ ]:
# make sure there is somewhere to save results
from datetime import date
today = date.today()
results_recognizer_location = os.path.join(RECORDINGS, 'results_recognizer_'+str(today))
if not os.path.exists(results_recognizer_location):
    os.makedirs(results_recognizer_location)

# Run the Ovenbird recognizer on your recordings

The following script will run the audio files in your "RECORDINGS" folder through the Ovenbird recognizer and export the results for each recording file.  

In [ ]:
def highpass_filter(y, fc, sr):
    nyq = 0.5 * sr
    b, a = butter(4, fc/nyq, 'high')
    return filtfilt(b, a, y)

def normalise_audio(y, sr=16000):
    y = highpass_filter(y=y, fc=500.0, sr=sr)
    y -= np.mean(y)
    return y

def spectrogram_from_audio(y, sr):
    sr = 16000
    n_fft = 128
    hop_length = 64
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=np.blackman)[10:,:] # remove DC
    D = librosa.power_to_db(np.abs(D)**2, ref=1.0)
    D = np.swapaxes(D,0,1)[np.newaxis,np.newaxis,:,:] # [batch,height,width,channels]
    return D

def process_directory(recordings):
    # find tensorflow model files
    meta_graph_path = 'oven-model.meta'
    checkpoint_path = 'oven-model'

    # list all audio files in directory
    audio_files = librosa.util.find_files(recordings, ext=['flac', 'wav'], recurse=True, case_sensitive=False, limit=None, offset=0)
    audio_files = list(set(audio_files)) # remove duplicates - not sure why find_files produces them!

    tf.compat.v1.reset_default_graph()
    with tf.compat.v1.Session() as sess:
        new_saver = tf.compat.v1.train.import_meta_graph(meta_graph_path)
        new_saver.restore(sess, checkpoint_path)
        features = tf.compat.v1.get_collection('input')[0]
        is_training = tf.compat.v1.get_collection('is_training')[0]
        level = tf.compat.v1.get_collection('level')[0]
        detection_fn = tf.compat.v1.get_collection('detection_fn')[0]

        for filename in tqdm_notebook(audio_files):
            y, sr = librosa.load(filename, sr=16000, mono=True)
            y = normalise_audio(y)
            S = spectrogram_from_audio(y, sr)
            level_dB, detection_function = sess.run([level, detection_fn], feed_dict={features: S, is_training: False})
            level_dB = level_dB[0,0,:,0]
            detection_function = detection_function[0,0,:,0]
            detection_function = expit(detection_function)
            duration_secs = y.size / float(sr)
            times = np.interp(np.arange(detection_function.size), [0,detection_function.size-1], [0,duration_secs])
            df = pd.DataFrame({'detection_window_start_time': times, 'detector_score': detection_function, 'relative_sound_level': level_dB})

            base = os.path.basename(filename)
            filename = os.path.splitext(base)[0]
            savename = os.path.join(results_location, filename + '_detection_function.csv')
            df.to_csv(savename, header=True, index=False)

process_directory(RECORDINGS)

# Ovenbird Recognizer Results

Results files will be exported in csv format to the folder called "results_recognizer_(date)" within the RECORDINGS folder to keep the results and the recordings together. The files will have the same name as the audio recordings files with "_detection_function" as a suffix. Each row in the files will consist of the detection window start time, the recognizer score (between 0 and 1) for that detection window, and the relative sound level during that detection window.

---
---

# Distance Estimation

In addition to predictions about the presence of Ovenbirds, predictions about the distance (in meters) of the bird from the ARU can be made based on the relative sound level of the signal in the recording.

**NOTE:** The distance estimation model provided here was developed using calibration recordings from SM3 ARUs (WildLife Acoustics, Concord, MA, USA) in mixedwood habitat. Distance estimates from this model for recordings collected under different conditions may be inaccurate because relative sound level can be affected by density of vegetation around the ARU and ARU recording/microphone settings. Additional details on the field methods and statistical modelling methods can be found in [Wilson and Bayne 2018](https://doi.org/10.5751/ACE-01248-130204) and [Yip et al. 2019](https://zslpublications.onlinelibrary.wiley.com/doi/full/10.1002/rse2.118).

This process that follows will produce distance estimates for Ovenbirds in recordings that were run through the recognizer above. It includes 95% confidence intervals, bounded by the values in the "distance_CI975" and "distance_CI025" fields.


## Setting up the data for the Distance Estimation model

If you want to estimate the distance of Ovenbirds in the dataset that you have run through the recognizer above then you can use the following steps to compare the results of the detector to the distance estimation model developed in Yip et al. 2019.

First you need to designate where the results are that the distance estimator will use to estimate distance. If you are running the distance estimator right after you run the recognizer above then you don't have to define the recognizer results folder because it was defined in the steps of running the recognizer. But if you are running the distance estimator during a different session then you will need to define the location of the recognizer results folder with the script below.


In [ ]:
# This doesn't need to be run if the results_recognizer_location folder was defined in the recognizer script above.  Change this folder path to that of your results folder.
results_recognizer_location = "C:/Users/User/Documents/Repos/Recognizer_Ovenbird/Recordings/results_recognizer_20240101"

Create a list of the csv files in the results_recognizer_location folder for the distance estimator to be run on.

In [ ]:
results_recognizer_files = [os.path.join(results_recognizer_location, f) for f in os.listdir(results_recognizer_location) if f.endswith('.csv')]

If you want to check that your recognizer results folder contains the csv's you expect then you can run the code below to see a list of all csv files in that folder.

In [ ]:
results_recognizer_files

### Read in the csv files to a dataframe
And make some adjustments to the columns for running through the distance estimator.

In [ ]:
# Read each csv into a separate dataframe
individual_dataframes = []
for file in results_recognizer_files:
    df = pd.read_csv(file)
    df['filename'] = os.path.basename(file)  # Add a new column with the recognizer result file name
    df = df.map(lambda x: x.replace('_detection_function.csv', '') if isinstance(x, str) else x) # Remove filename additions added in recognizer step
    df['species'] = 'OVEN' #Add species to dataframe
    df['level'] = df['relative_sound_level'].round(1) # Round the relative sound level to a single decimal place
    individual_dataframes.append(df)

# Combine individual dataframes into one large dataframe to run through the distance estimator
distance_input = pd.concat(individual_dataframes, ignore_index=True)

### Import the Distance Predictions file from the generalized linear model.

In [ ]:
predictions_model = pd.read_csv("DistancePredictions.csv")

## Generate Distance Predictions

This step takes the relative sound level from the classifier results and uses the generalized linear mixed model developed in Yip et al. 2019 to get an estimate of the distance of the Ovenbird calls in the dataset from the ARU.

Match the sound level in the results dataframe to the predictions file to generate the distance estimates for each detection window.


In [ ]:
distance_output = distance_input.merge(predictions_model, on=['level', 'species'], how='left')
distance_output = distance_output[['filename', 'detection_window_start_time', 'species', 'detector_score', 'relative_sound_level', 'distance', 'distance_CI975', 'distance_CI025']]

## Export distance estimate results

You can export the results of the distance estimation as one large csv of all results that were fed in, or split out as individual csv files; one for each file that was fed through the distance estimation process.

Since you may be generating distance estimation results on multiple recording files at the same time you can either export the results as one large csv file with the name of the classifier results files (if you used the recognizer steps above these should be the same as the original recording filename) as a field in the csv, or you can export individual csv files that are named to match the name of the classifier results files that were imported to the function (again, if you used the classifier steps above these should retain the name of the original recording file).

### Create the folder to store the distance estimation results.

In [ ]:
from datetime import date                   
today = date.today()                       
results_distance_location = os.path.join(RECORDINGS, 'results_distance_'+str(today))
os.makedirs(results_distance_location, exist_ok=True)

### 1. Single file output
This step produces a single, combined results csv named "combined_distance_function.csv" with the name of the individual classifer results files as a field for each row within the csv.

In [ ]:
distance_output.to_csv(os.path.join(results_distance_location, 'combined_distance_function.csv'), index=False)

### 2. Multiple file output

This step produces individual results files named after the individual results csv that the distance estimates relate to.

In [ ]:
split_field = 'filename'
unique_values = distance_output[split_field].unique()

for value in unique_values:
    split_df = distance_output[distance_output[split_field] == value].drop(columns=[split_field])
    output_file = os.path.join(results_distance_location, f'{value}'+ '_distance_function.csv')
    split_df.to_csv(output_file, index=False)

# Note on Distance Estimation Interpretation

The results of the distance estimation process are dependent on the relative sound levels calculated by the recognizer but they do not consider whether the detection window in question actually contains an Ovenbird.  The distance estimator will estimate a distance for all detection windows regardless of the recognizer score, so users of the distance estimation values should make sure that they only use distance estimates from detection windows that have classifier scores above their calculated threshold for positive identification.

In addition, several consecutive detection windows may have high recognizer scores, likely indicating that the Ovenbird song overlapped with several detection windows.  The distance estimation results will vary between these detection windows if the relative sound level in these detections windows varied (very likely), so in the case of consecutive detection windows with apparent Ovenbird detections you can take the lowest (closest) distance estimation value from these consecutive detection windows to determine how close the Ovenbird was.

# References:
Knight, E. C., K. C. Hannah, G. Foley, C. Scott, R. Mark Brigham, and E. M. Bayne. 2017. Recommendations for acoustic recognizer performance
assessment with application to five common automated signal recognition programs. Avian Conservation and Ecology 12(2):14.

Wilson, S. J., and E. M. Bayne. (2018). Use of an acoustic location system to understand how presence of conspecifics and canopy cover influence Ovenbird (*Seiurus aurocapilla*) space use near reclaimed wellsites in the boreal forest of Alberta. Avian Conservation and Ecology 13(2):4.

Yip, D. A., E. C. Knight, E. Haave‐Audet, S. J. Wilson, C. Charchuk, C. D. Scott, P. Solymos & E. M. Bayne. (2020). Sound level measurements from audio recordings provide objective distance estimates for distance sampling wildlife populations. Remote Sensing in Ecology and Conservation, 6(3), 301-315.